<a href="https://colab.research.google.com/github/EvanWAppel/work-examples/blob/main/loomisInvoiceComparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files 
import plotly.express as px
import datetime as datetime
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 800)

In [ ]:
loomis = pd.read_csv("loomis.csv", encoding = 'unicode_escape')
decreased = pd.read_csv("decreased.csv")
paused = pd.read_csv("paused.csv")

In [ ]:
# LIDs and totals

loom = loomis[['DESC_CODE','AMT','TAX','FURTHER_DESC1','FURTHER_DESC2']]
loom = loom.assign(total = loom['AMT'] + loom['TAX'])
loom = loom.rename(columns={'DESC_CODE':'description',
                                'FURTHER_DESC1':'date',
                                'FURTHER_DESC2':'lid'})

print(loom.head())
loo = loom.groupby(['lid'],as_index=False).agg({'total':sum,'date':np.size})
loo = loo.assign(visits=loo['date']/3)
loo = loo[['lid','total','visits']]
print(loo.head())

        description   AMT  TAX       date         lid  total
0  ATM DEPOSIT PULL  30.0  0.0  5/20/2022      101629   30.0
1          FUEL FEE   6.3  0.0  5/20/2022      101629    6.3
2     INSURANCE FEE   2.1  0.0  5/20/2022      101629    2.1
3  ATM DEPOSIT PULL  30.0  0.0   5/4/2022      101631   30.0
4          FUEL FEE   6.3  0.0   5/4/2022      101631    6.3
          lid   total  visits
0      101415  179.20     4.0
1      101416  115.20     3.0
2      101428   41.56     1.0
3      101436  307.20     8.0
4      101444  331.88     4.0


In [ ]:

# Paused list
paus = paused[['Date Sent','LID']]
paus = paus.rename(columns={'Date Sent':'date paused', 'LID':'lid'})
paus = paus.sort_values(by=['date paused'])
paus = paus.drop_duplicates(keep='last')
paus['lid'] = paus['lid'].astype(str)
print(paus.head())
print(paus.dtypes)

     date paused     lid
1373   2/15/2022  118013
67     2/15/2022  136305
66     2/15/2022  136223
65     2/15/2022  135122
64     2/15/2022  130635
date paused    object
lid            object
dtype: object


In [ ]:
# Decreased list
decr = decreased[['Date Requested','LID','Current Days','New Days']]
decr['LID'] = decr['LID'].astype(str)
decr['LID'] = decr['LID'].str.replace('\.0','')
decr = decr.rename(columns={'LID':'lid'})
decr = decr.assign(newcount=decr['New Days'].str.count(',') + 1)

print(decr.head())
print(decr.dtypes)


  Date Requested     lid Current Days New Days  newcount
0       2/2/2022  130479          T,F        T       1.0
1       2/2/2022  134821          T,F        T       1.0
2       2/2/2022  126248          T,F        T       1.0
3       2/2/2022  101429          T,F        T       1.0
4       2/2/2022  101430        M,W,F      M,F       2.0
Date Requested     object
lid                object
Current Days       object
New Days           object
newcount          float64
dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing

In [ ]:
loo['lid'] = loo['lid'].str.extract('(\d+)')
paus['lid'] = paus['lid'].str.extract('(\d+)')
decr['lid'] = decr['lid'].str.extract('(\d+)')
table = pd.merge(loo,paus,on='lid',how='left')
table = pd.merge(table,decr,on='lid',how='left')
table = table.sort_values(by=['Date Requested'])
table = table.drop_duplicates(subset=['lid'],keep='last')
print(table.head())

         lid   total  visits date paused Date Requested Current Days New Days  newcount
834   118016  153.60     4.0    3/7/2022       2/2/2022          T,F        T       1.0
1228  122704  335.10     5.0         NaN       2/8/2022          T,F        T       1.0
1488  124710  179.20     4.0   2/24/2022       2/8/2022          M,W        M       1.0
1458  124550  614.40     4.0   2/24/2022       2/8/2022         M,SA        M       1.0
1441  124428  110.84     1.0    3/7/2022       2/8/2022         M,SA        M       1.0


In [ ]:
table.to_csv('loomis_reconciliation.csv')
files.download('loomis_reconciliation.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>